# Load and Prepare data -> get staypoints and daily vehicle count
- consecutive trips ( represented by **tripcount** in input csv) within a day are assigned same trip_code ( else different trip_code)
- trip_code is used for merging trips i.e. merge consecutive trips with same trip_code as a single output row 
- remove entries with stay_time < threshold_in_sec ( e.g. 1800 seconds i.e. 30 minutes)
- Sample Input:
    -  ./new_data/gpsdata_202001-03.csv
- Sample Output:
    - ./new_data/output/gpsdata_202001-03_staypoints.csv
    - ./new_data/output/gpsdata_202001-03_dailyvehicle.csv


In [11]:
import pandas as pd
from datetime import datetime

#trip_code: consecutive trips within a day is assigned same trip_code ( else different trip code)
# It is used for merging trips i.e. merge consecutive tripcount as a single output row 
def assign_trip_code(df_single_ap):
    df_single_ap['trip_code']=None
    df_single_ap = df_single_ap.reset_index(drop=True)
    date_old = 0
    trip_count_old = 0
    trip_code = 0
    
    for idx,row in df_single_ap.iterrows():
        #print(idx)
        cur_trip_count = df_single_ap.iloc[idx].trip_prevs
        cur_date = df_single_ap.iloc[idx].date
        #print(cur_trip_count, trip_count_old, trip_code)
        
        if (trip_count_old ) == cur_trip_count:
            df_single_ap.at[idx,'trip_code'] = str(trip_code)
        elif ( ( (trip_count_old +1 ) == cur_trip_count) ):
            if date_old == cur_date:
                df_single_ap.at[idx,'trip_code'] =  str(trip_code)            
            else:
                trip_code += 1
        else:
            trip_code += 1
            df_single_ap.at[idx,'trip_code'] =  str(trip_code)

        trip_count_old = cur_trip_count
        date_old = cur_date

    return df_single_ap 

def read_input_data(csv_file):
    df = pd.read_csv(csv_file,usecols=['serial','tripid','tripcount','tlm_datagettime','lat','lon'])
    #df = pd.read_csv(csv_file)#,usecols=['serial','tripid','tripcount','tlm_datagettime','lat','lon'])

    df.rename(columns = {'serial':'ap_id','tlm_datagettime':'timestamp'}, inplace = True)
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df=df.sort_values(by=['timestamp'])
    df['date'] = pd.to_datetime(df['timestamp']).dt.date
    return df


def prepare_trip_summary(df):# summarize each trip
    
    arr_ap_ids = df.ap_id.unique()

    arr_trip_summary = []
    for ap_id in arr_ap_ids:
        
        df_single_ap = df.query("ap_id=='"+ap_id+"'").copy()
        df_single_ap = df_single_ap.sort_values(by=['timestamp'])
        arr_trips = df_single_ap.tripcount.unique()
        '''
        if ap_id=='AP520040':
            #display(df_single_ap)
            print (arr_trips)
        '''
        #trip_count = arr_trips[0]
        #trip_count
        ts_prevs = df_single_ap.timestamp.min()
        lat_prevs = df_single_ap.iloc[0].lat
        lon_prevs = df_single_ap.iloc[0].lon

        trip_count_prevs = -9 #  small number (far below real trip_count value) for initialization purpose
        for trip_count in arr_trips:

            # process consicutive trips only e.g. trip_count_prevs = 12 and  trip_count=13
            df_single_trip = df_single_ap.query("tripcount=='"+str(trip_count)+"'").copy()
            df_single_trip = df_single_trip.sort_values(by=['timestamp'])
            max_ts = df_single_trip.timestamp.max()
            min_ts= df_single_trip.timestamp.min()

            stay_time =  min_ts - ts_prevs
            trip_time = (max_ts - min_ts).total_seconds()

            lat_min = df_single_trip.iloc[0].lat
            lon_min = df_single_trip.iloc[0].lon
            lat_max = df_single_trip.iloc[len(df_single_trip)-1].lat 
            lon_max = df_single_trip.iloc[len(df_single_trip)-1].lon

            avg_lat = (lat_prevs + lat_min)/2
            avg_lon = (lon_prevs + lon_min)/2

            lat_prevs = lat_max
            lon_prevs = lon_max

            if ( trip_count_prevs  == trip_count-1):
                
                '''
                if ap_id=='AP520040':
                    print (trip_count_prevs, trip_count,' -- ', min_ts.date(), ts_prevs.date())
                '''         
                if min_ts.date() != ts_prevs.date():
                    trip_count_prevs = trip_count
                    ts_prevs = max_ts
                    #print ('SKIP: ', min_ts,ts_prevs)                    
                    continue
                    
                #if ap_id=='AP520040':
                    #print ( '\t' ,trip_count_prevs, trip_count,' -- ', min_ts.date(), ts_prevs.date())
                    
                arr_trip_summary.append({
                    'ap_id': ap_id,

                    'trip_prevs': trip_count_prevs,
                    'trip_count': trip_count,
                    'ts_prevs': ts_prevs,
                    'ts_min': min_ts,
                    'stay_time': stay_time.total_seconds(),
                    'avg_lon_with_prvs': avg_lon,
                    'avg_lat_with_prvs': avg_lat,
              
                    })

            trip_count_prevs = trip_count
            ts_prevs = max_ts

    trip_df =  pd.DataFrame(arr_trip_summary)      
    trip_df['date'] = trip_df['ts_min'].dt.date

    return trip_df


# if multiple consecutive trips in a single day then merge them as single one
def merge_consecutive_trips_in_single_day(trip_df, threshold_in_sec, final_csv):
    
    arr_ap_ids = trip_df.ap_id.unique()
    arr_trip_merged = []
    for ap_id in arr_ap_ids:
        df_single_ap = trip_df.query("ap_id=='"+ap_id+"'").copy()
        
        df_single_ap = assign_trip_code(df_single_ap)
        #display(df_single_ap)

        arr_trip_code = df_single_ap.trip_code.unique()

        for trip_code in arr_trip_code:
            df_trip_code = df_single_ap.query("trip_code=='"+str(trip_code)+"'")

            arr_trip_merged.append( {
                        'ap_id': ap_id,
                        'date':df_trip_code.date.min(),
                        'trip_prevs': df_trip_code.trip_prevs.min(),
                        'trip_count': df_trip_code.trip_count.max(),
                        'ts_prevs_stop': df_trip_code.ts_prevs.min(),
                        'timestamp': df_trip_code.ts_min.max(), # timnestamp of car starting 
                        'stay_time': df_trip_code.stay_time.sum(),
                        'lon': df_trip_code.avg_lon_with_prvs.mean(), # avg_lon_with_prvs
                        'lat':  df_trip_code.avg_lat_with_prvs.mean(), # avg_lat_with_prvs
                        'trip_code': trip_code
                        } )

    df_final =  pd.DataFrame(arr_trip_merged) 
    
    # remove entries with stay_time < threshold_in_sec ( e.g. 1800 seconds)
    df_final = df_final[df_final['stay_time']>=threshold_in_sec]
    df_final.to_csv(final_csv, index=False) 
    
    return df_final

In [12]:
threshold_in_sec = 1800 # # remove entries with stay_time < 1800 seconds
#input_csv='data/2019-04_2019-08_GyokuSendo.csv'
#final_csv='data/2019-04_2019-08_GyokuSendo_staypoints_2.csv'
'''
input_csv='new_data/gpsdata_202001-03.csv'
final_csv='new_data/output/gpsdata_202001-03_staypoints.csv'
daily_vehicle_csv='new_data/output/gpsdata_202001-03_dailyvehicle.csv'
'''
input_csv= 'new_data/gpsdata_201803-12.csv'
final_csv=  'new_data/output/gpsdata_201803-12_staypoints.csv'
daily_vehicle_csv='new_data/output/gpsdata_201803-12_dailyvehicle.csv'


df = read_input_data( input_csv )
trip_df = prepare_trip_summary(df)
df_final = merge_consecutive_trips_in_single_day(trip_df, threshold_in_sec, final_csv)
#print(len(trip_df), len(df_final))

In [13]:
#df = df.query("ap_id=='AP622732'") 

# Count and save daily unique vehicles

In [14]:
df_daily_unique_ap = df_final[['date','ap_id']]
df_daily_unique_ap = df_daily_unique_ap.drop_duplicates(['date','ap_id'])
df_daily=df_daily_unique_ap[['date','ap_id']].groupby('date').agg(['count']).reset_index()
df_daily.to_csv(daily_vehicle_csv ,  index = False)

# Examine result and inoutdata

In [17]:
trip_df1 = df_final.query("ap_id=='AP622732'")
trip_df1.head(10)

,ap_id,date,trip_prevs,trip_count,ts_prevs_stop,timestamp,stay_time,lon,lat,trip_code
6369,AP622732,2018-04-15,602,605,2018-04-15 16:02:14,2018-04-15 17:39:21,5825.0,127.748504,26.141682,1
6370,AP622732,2018-04-16,615,616,2018-04-16 15:34:11,2018-04-16 16:59:10,5099.0,127.747982,26.141301,2
6371,AP622732,2018-05-20,877,878,2018-05-20 14:01:13,2018-05-20 16:42:19,9666.0,127.747158,26.140155,3
6372,AP622732,2018-05-28,954,955,2018-05-28 11:26:27,2018-05-28 14:30:49,11062.0,127.749294,26.141307,4
6373,AP622732,2018-06-02,984,985,2018-06-02 11:30:02,2018-06-02 13:56:02,8760.0,127.748537,26.141290,5
6374,AP622732,2018-06-05,1011,1013,2018-06-05 14:05:03,2018-06-05 16:58:29,10277.0,127.748348,26.141099,6
6375,AP622732,2018-06-19,1141,1142,2018-06-19 11:03:36,2018-06-19 12:14:35,4259.0,127.749403,26.141242,7
6376,AP622732,2018-06-26,1202,1203,2018-06-26 10:26:13,2018-06-26 14:04:52,13119.0,127.749361,26.141330,8
6377,AP622732,2018-07-01,1258,1259,2018-07-01 11:18:03,2018-07-01 14:46:27,12504.0,127.749516,26.141373,9
6378,AP622732,2018-07-21,1373,1374,2018-07-21 15:18:18,2018-07-21 16:49:13,5455.0,127.749085,26.141354,10


In [1]:
df.query("ap_id=='AP622732' and  tripcount >400")#.query("tripcount >4000")

NameError: name 'df' is not defined